### Modifying PASCAL VOC labels

This notebook shows how to split the images into a grid and modify the existing labels. This is needed when using pre-trained CNN's which are trained on images of size relatively smaller than those in the dataset. 

While detecting smaller objects, resizing the large images into a small input size will make these smaller objects loose their features. Hence we divide the large images into 2 by 2 or 3 by 3 grid and generate new annotation files with modified coordinates. 

note: Overlapping annotations between two partitions are logged.


In [2]:
import numpy as np 
import matplotlib as mp
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import misc
import sys
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, SubElement
from lxml import etree
import os

In [4]:

XML_EXT = '.xml'
ENCODE_METHOD = 'utf-8'

class PascalVocReader:

    def __init__(self, filepath, q_y, q_x, h, w):
        # shapes type:
        # [labbel, [(x1,y1), (x2,y2), (x3,y3), (x4,y4)], color, color, difficult]
        self.shapes = []
        self.filepath = filepath
        self.verified = False
        self.q_x = q_x
        self.q_y = q_y
        self.h = h
        self.w = w
        self.x_origin = q_x * w
        self.y_origin = q_y * h
        self.overlap = False
        print ("origin"+str(self.x_origin) + "," + str(self.y_origin))

    def parseXML(self):
        assert self.filepath.endswith(XML_EXT), "Unsupport file format"
        parser = etree.XMLParser(encoding=ENCODE_METHOD)
        tree = ElementTree.parse(self.filepath, parser=parser)
        xmltree = tree.getroot()
        filename = xmltree.find('filename')
        old_filename = filename.text
        filename.text = old_filename.replace(".jpg", "_"+str(self.q_x)+"_"+str(self.q_y)+".jpg") #new filename
        
        path = xmltree.find('path')
        old_path = path.text
        path.text = old_path.replace(".jpg", "_"+str(self.q_x)+"_"+str(self.q_y)+".jpg")
        
        size = xmltree.find('size')
        width = size.find('width')
        height = size.find('height')
        width.text = str(self.w)
        height.text = str(self.h)
               

        for object_iter in xmltree.findall('object'):
            bndbox = object_iter.find("bndbox")
            label = object_iter.find('name').text
            
            xmin = int(bndbox.find('xmin').text) 
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            print (str(xmin) +","+str(xmax)+","+str(ymin)+","+str(ymax))
            if xmin >= self.x_origin and xmax <= (self.x_origin + self.w) and ymin >= self.y_origin and ymax <= (self.y_origin + self.h):
                bndbox.find('xmin').text = str(xmin - self.x_origin)
                bndbox.find('xmax').text = str(xmax - self.x_origin)
                bndbox.find('ymin').text = str(ymin - self.y_origin)
                bndbox.find('ymax').text = str(ymax - self.y_origin)
                print ("bdbox accepted n"+str(self.q_x) + str(self.q_y))
            else:
                print ("bnbox removed in"+str(self.q_x) + str(self.q_y))
                if label == "fish":
                    self.overlap = True
                xmltree.remove(object_iter)
        if len(xmltree.findall('object')) != 0:
            print("executed")
            new_filepath = self.filepath.replace(".xml", "_"+str(self.q_x)+"_"+str(self.q_y)+".xml")
            tree.write(new_filepath)
            return True
        else:
            return False



def modify_labeldata(path,gridSize=3):
    #set the directory paths
    image_dir = os.path.join(path, "testing")
    anno_dir = os.path.join(path, "test_annotations")
    
    H=0
    W=0
    c=0
    H_out = 0
    W_out = 0
    anno_list = os.listdir(anno_dir)
    #list the images and annotations
    for i,file in enumerate(os.listdir(image_dir)):
        if(".jpg" in file):
            img = misc.imread(os.path.join(image_dir,file))
            if i==0:
                H, W,c = img.shape
                H_out = H//gridSize
                W_out = W//gridSize
            overlap = False
            for j in range(gridSize):  # rows
                for k in range(gridSize): #columns
                    x_origin = j * W_out
                    y_origin = k * H_out
                    img_crop = img[x_origin : x_origin + H_out , y_origin : y_origin + W_out]
                    # read the .xml and get the objects in it
                    anno_name = file.replace(".jpg", ".xml")
                    if anno_name in anno_list:
                        # for each object
                        reader = PascalVocReader(os.path.join(anno_dir,anno_name), j, k, H_out,W_out)
                        res = reader.parseXML()
                        if reader.overlap:
                            overlap = True
                        if res :
                            new_imgpath = file.replace(".jpg", "_"+str(k)+"_"+str(j)+".jpg")
                            misc.imsave(os.path.join(image_dir,new_imgpath),img_crop)
            if overlap:
                if not os.path.exists(os.path.join(path,"log")):
                    os.makedirs(os.path.join(path,"log"))
                os.rename(os.path.join(image_dir,file),os.path.join(path,"log",file))


modify_labeldata("/home/pushyamik/Desktop/labeldata", 2)

/home/pushyamik/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:86: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/pushyamik/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:107: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


origin0,0
1899,2040,386,518
bnbox removed in00
1537,1693,1359,1465
bnbox removed in00
origin1024,0
1899,2040,386,518
bdbox accepted n10
1537,1693,1359,1465
bnbox removed in10
executed
origin0,1024
1899,2040,386,518
bnbox removed in01
1537,1693,1359,1465
bnbox removed in01
origin1024,1024
1899,2040,386,518
bnbox removed in11
1537,1693,1359,1465
bdbox accepted n11
executed
origin0,0
819,969,636,777
bdbox accepted n00
executed
origin1024,0
819,969,636,777
bnbox removed in10
origin0,1024
819,969,636,777
bnbox removed in01
origin1024,1024
819,969,636,777
bnbox removed in11
origin0,0
14,249,1521,1765
bnbox removed in00
origin1024,0
14,249,1521,1765
bnbox removed in10
origin0,1024
14,249,1521,1765
bdbox accepted n01
executed
origin1024,1024
14,249,1521,1765
bnbox removed in11
origin0,0
28,140,950,1086
bnbox removed in00
1014,1122,492,630
bnbox removed in00
1575,1719,830,953
bnbox removed in00
1708,1819,539,639
bnbox removed in00
555,684,50,162
bdbox accepted n00
executed
origin1024,0
28,140,9

origin1024,1024
858,1122,212,421
bnbox removed in11
519,649,762,942
bnbox removed in11
399,534,1112,1289
bnbox removed in11
1169,1369,630,748
bnbox removed in11
1028,1169,921,1068
bnbox removed in11
1617,1788,1,92
bnbox removed in11
origin0,0
955,1105,668,774
bnbox removed in00
origin1024,0
955,1105,668,774
bnbox removed in10
origin0,1024
955,1105,668,774
bnbox removed in01
origin1024,1024
955,1105,668,774
bnbox removed in11
origin0,0
64,261,503,697
bdbox accepted n00
390,687,489,603
bdbox accepted n00
executed
origin1024,0
64,261,503,697
bnbox removed in10
390,687,489,603
bnbox removed in10
origin0,1024
64,261,503,697
bnbox removed in01
390,687,489,603
bnbox removed in01
origin1024,1024
64,261,503,697
bnbox removed in11
390,687,489,603
bnbox removed in11
origin0,0
1884,2048,1739,1943
bnbox removed in00
origin1024,0
1884,2048,1739,1943
bnbox removed in10
origin0,1024
1884,2048,1739,1943
bnbox removed in01
origin1024,1024
1884,2048,1739,1943
bdbox accepted n11
executed
origin0,0
987,112

origin0,0
1625,1764,924,1050
bnbox removed in00
608,743,545,662
bdbox accepted n00
executed
origin1024,0
1625,1764,924,1050
bnbox removed in10
608,743,545,662
bnbox removed in10
origin0,1024
1625,1764,924,1050
bnbox removed in01
608,743,545,662
bnbox removed in01
origin1024,1024
1625,1764,924,1050
bnbox removed in11
608,743,545,662
bnbox removed in11
origin0,0
1225,1490,1789,1950
bnbox removed in00
origin1024,0
1225,1490,1789,1950
bnbox removed in10
origin0,1024
1225,1490,1789,1950
bnbox removed in01
origin1024,1024
1225,1490,1789,1950
bdbox accepted n11
executed
origin0,0
978,1140,812,962
bnbox removed in00
origin1024,0
978,1140,812,962
bnbox removed in10
origin0,1024
978,1140,812,962
bnbox removed in01
origin1024,1024
978,1140,812,962
bnbox removed in11
origin0,0
1484,1708,118,233
bnbox removed in00
origin1024,0
1484,1708,118,233
bdbox accepted n10
executed
origin0,1024
1484,1708,118,233
bnbox removed in01
origin1024,1024
1484,1708,118,233
bnbox removed in11
origin0,0
975,1175,727,94

origin1024,1024
822,961,1683,1853
bnbox removed in11
1914,2048,1137,1259
bdbox accepted n11
executed
origin0,0
1311,1452,1527,1692
bnbox removed in00
origin1024,0
1311,1452,1527,1692
bnbox removed in10
origin0,1024
1311,1452,1527,1692
bnbox removed in01
origin1024,1024
1311,1452,1527,1692
bdbox accepted n11
executed
origin0,0
684,825,1518,1627
bnbox removed in00
origin1024,0
684,825,1518,1627
bnbox removed in10
origin0,1024
684,825,1518,1627
bdbox accepted n01
executed
origin1024,1024
684,825,1518,1627
bnbox removed in11
origin0,0
855,1155,1697,1953
bnbox removed in00
origin1024,0
855,1155,1697,1953
bnbox removed in10
origin0,1024
855,1155,1697,1953
bnbox removed in01
origin1024,1024
855,1155,1697,1953
bnbox removed in11
origin0,0
602,725,1050,1171
bnbox removed in00
origin1024,0
602,725,1050,1171
bnbox removed in10
origin0,1024
602,725,1050,1171
bdbox accepted n01
executed
origin1024,1024
602,725,1050,1171
bnbox removed in11
origin0,0
284,575,1262,1503
bnbox removed in00
1455,1875,156

...

origin0,0
1308,1419,509,656
bnbox removed in00
1278,1558,74,262
bnbox removed in00
origin1024,0
1308,1419,509,656
bdbox accepted n10
1278,1558,74,262
bdbox accepted n10
executed
origin0,1024
1308,1419,509,656
bnbox removed in01
1278,1558,74,262
bnbox removed in01
origin1024,1024
1308,1419,509,656
bnbox removed in11
1278,1558,74,262
bnbox removed in11
origin0,0
1246,1449,1086,1209
bnbox removed in00
origin1024,0
1246,1449,1086,1209
bnbox removed in10
origin0,1024
1246,1449,1086,1209
bnbox removed in01
origin1024,1024
1246,1449,1086,1209
bdbox accepted n11
executed
origin0,0
1711,1825,771,892
bnbox removed in00
origin1024,0
1711,1825,771,892
bdbox accepted n10
executed
origin0,1024
1711,1825,771,892
bnbox removed in01
origin1024,1024
1711,1825,771,892
bnbox removed in11
origin0,0
169,340,1368,1600
bnbox removed in00
1264,1369,518,647
bnbox removed in00
origin1024,0
169,340,1368,1600
bnbox removed in10
1264,1369,518,647
bdbox accepted n10
executed
origin0,1024
169,340,1368,1600
bdbox acce

origin0,0
969,1178,259,389
bnbox removed in00
234,375,1765,1980
bnbox removed in00
origin1024,0
969,1178,259,389
bnbox removed in10
234,375,1765,1980
bnbox removed in10
origin0,1024
969,1178,259,389
bnbox removed in01
234,375,1765,1980
bdbox accepted n01
executed
origin1024,1024
969,1178,259,389
bnbox removed in11
234,375,1765,1980
bnbox removed in11
origin0,0
305,411,1827,2048
bnbox removed in00
origin1024,0
305,411,1827,2048
bnbox removed in10
origin0,1024
305,411,1827,2048
bdbox accepted n01
executed
origin1024,1024
305,411,1827,2048
bnbox removed in11
origin0,0
1681,1802,1259,1359
bnbox removed in00
origin1024,0
1681,1802,1259,1359
bnbox removed in10
origin0,1024
1681,1802,1259,1359
bnbox removed in01
origin1024,1024
1681,1802,1259,1359
bdbox accepted n11
executed
origin0,0
1299,1449,1289,1539
bnbox removed in00
1834,1975,1697,1830
bnbox removed in00
origin1024,0
1299,1449,1289,1539
bnbox removed in10
1834,1975,1697,1830
bnbox removed in10
origin0,1024
1299,1449,1289,1539
bnbox rem

origin0,0
1264,1499,286,421
bnbox removed in00
675,805,480,571
bdbox accepted n00
1114,1269,206,318
bnbox removed in00
737,1134,1348,1604
bnbox removed in00
102,240,203,339
bdbox accepted n00
executed
origin1024,0
1264,1499,286,421
bdbox accepted n10
675,805,480,571
bnbox removed in10
1114,1269,206,318
bdbox accepted n10
737,1134,1348,1604
bnbox removed in10
102,240,203,339
bnbox removed in10
executed
origin0,1024
1264,1499,286,421
bnbox removed in01
675,805,480,571
bnbox removed in01
1114,1269,206,318
bnbox removed in01
737,1134,1348,1604
bnbox removed in01
102,240,203,339
bnbox removed in01
origin1024,1024
1264,1499,286,421
bnbox removed in11
675,805,480,571
bnbox removed in11
1114,1269,206,318
bnbox removed in11
737,1134,1348,1604
bnbox removed in11
102,240,203,339
bnbox removed in11
origin0,0
467,778,42,212
bdbox accepted n00
executed
origin1024,0
467,778,42,212
bnbox removed in10
origin0,1024
467,778,42,212
bnbox removed in01
origin1024,1024
467,778,42,212
bnbox removed in11
origi

origin0,0
123,252,907,1033
bnbox removed in00
origin1024,0
123,252,907,1033
bnbox removed in10
origin0,1024
123,252,907,1033
bnbox removed in01
origin1024,1024
123,252,907,1033
bnbox removed in11
origin0,0
272,546,724,912
bdbox accepted n00
executed
origin1024,0
272,546,724,912
bnbox removed in10
origin0,1024
272,546,724,912
bnbox removed in01
origin1024,1024
272,546,724,912
bnbox removed in11
origin0,0
284,496,956,1092
bnbox removed in00
1208,1311,1092,1221
bnbox removed in00
658,834,1812,1927
bnbox removed in00
1581,1667,103,206
bnbox removed in00
1258,1352,183,292
bnbox removed in00
origin1024,0
284,496,956,1092
bnbox removed in10
1208,1311,1092,1221
bnbox removed in10
658,834,1812,1927
bnbox removed in10
1581,1667,103,206
bdbox accepted n10
1258,1352,183,292
bdbox accepted n10
executed
origin0,1024
284,496,956,1092
bnbox removed in01
1208,1311,1092,1221
bnbox removed in01
658,834,1812,1927
bdbox accepted n01
1581,1667,103,206
bnbox removed in01
1258,1352,183,292
bnbox removed in01


origin0,0
952,1278,421,695
bnbox removed in00
1469,1772,59,380
bnbox removed in00
origin1024,0
952,1278,421,695
bnbox removed in10
1469,1772,59,380
bdbox accepted n10
executed
origin0,1024
952,1278,421,695
bnbox removed in01
1469,1772,59,380
bnbox removed in01
origin1024,1024
952,1278,421,695
bnbox removed in11
1469,1772,59,380
bnbox removed in11
origin0,0
858,984,1089,1203
bnbox removed in00
origin1024,0
858,984,1089,1203
bnbox removed in10
origin0,1024
858,984,1089,1203
bdbox accepted n01
executed
origin1024,1024
858,984,1089,1203
bnbox removed in11
origin0,0
1867,2008,1186,1339
bnbox removed in00
origin1024,0
1867,2008,1186,1339
bnbox removed in10
origin0,1024
1867,2008,1186,1339
bnbox removed in01
origin1024,1024
1867,2008,1186,1339
bdbox accepted n11
executed
origin0,0
37,196,1256,1397
bnbox removed in00
1487,1628,1795,1924
bnbox removed in00
origin1024,0
37,196,1256,1397
bnbox removed in10
1487,1628,1795,1924
bnbox removed in10
origin0,1024
37,196,1256,1397
bdbox accepted n01
148